In [23]:
#Google Places API notebook

import pandas as pd
import requests
import os

In [ ]:
# Retreiving API key from environment variable
key = os.environ.get('api_key_places')

In [24]:
# For Sam's API key, comment out
from dotenv import load_dotenv
load_dotenv()
key = os.getenv('YELP_API_KEY3')

In [15]:
# For loop that pulls data for all hotels in Toronto using the Yelp API.  
# Yelp only allows 50 results at a time but we want 200 results, so we have to run the query 4 times with the offset parameter set to 0, 50, 100, and 150.

response = []

for offset in range(0,200,50):
    url = 'https://api.yelp.com/v3/businesses/search'
    params = {
        'limit': 50,
        'location': 'Toronto',
        'term': 'hotel',
        'categories': 'hotel',
        'offset': offset
    }

    headers = {"accept": "application/json","Authorization": 'bearer ' + key}
    response += requests.get(url, headers=headers, params=params).json()['businesses']

df_hotels = pd.json_normalize(response)

In [16]:
# Unnesting the 'categories column'
df_hotels['categories'] = pd.json_normalize(df_hotels['categories'].explode('categories'))['alias']

In [20]:
# Confirming dataframe shape
df_hotels.shape

(200, 24)

In [18]:
# Viewing sample of dataframe
df_hotels.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address
0,yeesT9W64tpVq5Gcae18Sg,delta-hotels-by-marriott-toronto-toronto-2,Delta Hotels by Marriott Toronto,https://s3-media2.fl.yelpcdn.com/bphoto/TRWtlS...,False,https://www.yelp.com/biz/delta-hotels-by-marri...,161,hotels,4.0,[],...,43.643069,-79.383865,75 Lower Simcoe Street,,,Toronto,M5J 3A6,CA,ON,"[75 Lower Simcoe Street, Toronto, ON M5J 3A6, ..."
1,kKnGY0Xj7dG4ubjrbygynw,residence-inn-by-marriott-toronto-downtown-tor...,Residence Inn by Marriott Toronto Downtown,https://s3-media4.fl.yelpcdn.com/bphoto/PFaLls...,False,https://www.yelp.com/biz/residence-inn-by-marr...,95,hotels,4.0,[],...,43.644790,-79.390530,255 Wellington Street W,,,Toronto,M5V 3P9,CA,ON,"[255 Wellington Street W, Toronto, ON M5V 3P9,..."
2,XHrPQKUTJXJGPH9etKUYUw,courtyard-by-marriott-toronto-downtown-toronto,Courtyard by Marriott Toronto Downtown,https://s3-media2.fl.yelpcdn.com/bphoto/d6S6ga...,False,https://www.yelp.com/biz/courtyard-by-marriott...,152,hotels,3.0,[],...,43.662410,-79.383149,475 Yonge Street,,,Toronto,M4Y 1X7,CA,ON,"[475 Yonge Street, Toronto, ON M4Y 1X7, Canada]"
3,zYRBRfYuq_6x-wNKa8NqrA,le-germain-hotel-toronto,Le Germain Hotel,https://s3-media3.fl.yelpcdn.com/bphoto/m1977c...,False,https://www.yelp.com/biz/le-germain-hotel-toro...,55,hotels,4.5,[],...,43.645751,-79.391025,30 Mercer Street,,,Toronto,M5V 1H3,CA,ON,"[30 Mercer Street, Toronto, ON M5V 1H3, Canada]"
4,PCkJAv7WQJ08C1RA6ilqWA,1-hotel-toronto-toronto,1 Hotel Toronto,https://s3-media4.fl.yelpcdn.com/bphoto/abOBDo...,False,https://www.yelp.com/biz/1-hotel-toronto-toron...,5,hotels,5.0,[],...,43.642826,-79.401439,550 Wellington Street W,None,,Toronto,M5V 2V4,CA,ON,"[550 Wellington Street W, Toronto, ON M5V 2V4,..."


In [27]:
# Converting prices from dollar signs to integers
df_hotels['price'] = df_hotels['price'].str.len()

In [22]:
# Writing dataframe to a .csv file
df_hotels.to_csv('yelp_hotels.csv',index=False)